<a href="https://colab.research.google.com/github/cabamarcos/SuperMask/blob/main/Prueba_pesos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler,ReduceLROnPlateau
from tensorflow.keras.datasets import mnist

import os

from keras.models import Sequential
from keras.layers import Dense


In [31]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")


X_train shape: (60000, 28, 28)
y_train shape: (60000,)
X_test shape: (10000, 28, 28)
y_test shape: (10000,)


In [32]:
datagen = ImageDataGenerator(rotation_range=10,  zoom_range = 0.1,  width_shift_range=0.1,  height_shift_range=0.1)

In [33]:
# Learning rate annealer

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',
                                patience=3,
                                verbose=1,
                                factor=0.2,
                                min_lr=1e-6)

In [34]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=3, activation='relu', padding='same', input_shape=[28, 28, 1]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding='same'),
    keras.layers.MaxPool2D(),
    keras.layers.Conv2D(32, kernel_size=3, activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding='same'),
    keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
X_train = X_train.reshape((60000, 28, 28, 1))

In [36]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.08)
history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=64),
                              epochs=1, validation_data=(X_valid, y_valid), callbacks=[reduce_lr])
print("Maximum Train Accuracy : {} Maximum Validation Accuracy : {}".format(max(history.history['accuracy']), max(history.history['val_accuracy'])))


<ipython-input-36-821ba362ef7b>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=64),


863/863 [==============================] - 290s 334ms/step - loss: 0.3732 - accuracy: 0.8827 - val_loss: 0.1009 - val_accuracy: 0.9712 - lr: 0.0010
Maximum Train Accuracy : 0.8826992511749268 Maximum Validation Accuracy : 0.9712499976158142


In [37]:
# Obtiene los pesos de cada capa
for i, layer in enumerate(model.layers):
    if layer.get_weights():  # Comprueba si la capa tiene pesos
        weights = layer.get_weights()
        print(f"Capa {i}:")
        print("Pesos:")
        print(weights)
    else:
        print(f"Capa {i}: Esta capa no tiene pesos.")

Capa 0:
Pesos:
[array([[[[-0.14580683,  0.10688996, -0.10104157,  0.00034035,
           0.09404492,  0.0343553 ,  0.04839827,  0.13868943,
          -0.03803222,  0.04628252,  0.00311111,  0.13530101,
           0.14450954,  0.07834459, -0.07812981,  0.06337897,
           0.02390683, -0.17172228,  0.1646039 , -0.09488536,
          -0.02237496,  0.15458658, -0.11887193,  0.01347665,
          -0.00046952,  0.02347698,  0.04888731, -0.03401876,
          -0.04043934, -0.03750772,  0.10304258, -0.12903342]],

        [[-0.01508601,  0.05049722, -0.0305905 ,  0.02657529,
           0.09526797,  0.13005817, -0.15039194,  0.02707781,
           0.0294665 , -0.11740563, -0.11727507,  0.12505513,
           0.11744808,  0.01556899, -0.00413289,  0.08076224,
          -0.02597469, -0.05919636, -0.0828467 , -0.03874431,
          -0.00992853,  0.12429047, -0.05945051,  0.01719841,
          -0.08828987, -0.09197547,  0.12302075,  0.07287564,
          -0.05448404,  0.13356838, -0.12915969,  0